# GIS Project "ITRF2020"

The tasks that are performed within this notebook:

1. Open the 4 files and keep only the stations that belongs to a site (look at the five first numbers of the DOMES (id) number) which hosts at least 3 instruments from 2 different measurement techniques (GNSS, DORIS, SLR or VLBI).

2. Calculate for each station the polygon of its extent (convex envelope) and export it as a SHP file (convert geometry from EPSG:4978 to EPSG:4326 at the begining).

3. List for each station the images (between 2022/01/01 and 2022/09/30) that are covering the extent and export it as a SHP file:
 - For each station, write a <station_id>.json temporary file when you do the API request;
 - Merge all information in one SHP file after.

In [19]:
#Import necessary libraries
import numpy as np
import pandas as pd
import scipy as sc
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import json
import shapely
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon

In [3]:
#Load all data and concatenate files
ITRF2020_DORIS_cart = pd.read_fwf('ipynb_checkpoints\\ITRF2020_DORIS_cart.txt')
ITRF2020_GNSS_cart = pd.read_fwf('ipynb_checkpoints\\ITRF2020_GNSS_cart.txt')
ITRF2020_SLR_cart = pd.read_fwf('ipynb_checkpoints\\ITRF2020_SLR_cart.txt')
ITRF2020_VLBI_cart = pd.read_fwf('ipynb_checkpoints\\ITRF2020_VLBI_cart.txt')
concatenated_files = pd.concat([ITRF2020_DORIS_cart, ITRF2020_GNSS_cart, ITRF2020_SLR_cart, ITRF2020_VLBI_cart])
concatenated_files

,id,name,type,code,x,y,z,dx,dy,dz
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,0.002,0.0025,0.002
1,10002S019,Grasse (OCA),DORIS,GR4B,4.581681e+06,5.561669e+05,4.389371e+06,0.0019,0.0024,0.0017
2,10003S001,Toulouse,DORIS,TLSA,4.628047e+06,1.196707e+05,4.372788e+06,0.0054,0.0062,0.0051
3,10003S003,Toulouse,DORIS,TLHA,4.628693e+06,1.199851e+05,4.372105e+06,0.0034,0.0042,0.0032
4,10003S005,Toulouse,DORIS,TLSB,4.628694e+06,1.199851e+05,4.372105e+06,0.0026,0.0039,0.0025
...,...,...,...,...,...,...,...,...,...,...
149,50243S001,Warkworth,VLBI,7377,-5.115324e+06,4.778433e+05,-3.767193e+06,0.0017,0.0012,0.0016
150,50505S003,KWAJALEIN ATOLL,VLBI,4968,-6.143536e+06,1.363999e+06,1.034708e+06,0.8689,0.2507,0.3272
151,59968S001,Katherine - Nor,VLBI,7375,-4.147355e+06,4.581542e+06,-1.573303e+06,0.0014,0.0014,0.0012
152,66006S004,Syowa,VLBI,7342,1.766194e+06,1.460411e+06,-5.932273e+06,0.0034,0.0031,0.0071


In [4]:
#Leave only 5 first symbols in id column in order to get station_id
concatenated_files['station_id'] = concatenated_files['id'].str[:5]

In [5]:
concatenated_files.sort_values('station_id')

,id,name,type,code,x,y,z,dx,dy,dz,station_id
1,10001S007,Paris,GNSS,OP71,4.202780e+06,1.713707e+05,4.778661e+06,0.001,0.0007,0.0011,10001
0,10001S006,Paris,GNSS,OPMT,4.202777e+06,1.713682e+05,4.778660e+06,0.0009,0.0007,0.0009,10001
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,0.002,0.0025,0.002,10002
0,10002M003,Grasse (OCA),VLBI,7605,4.581697e+06,5.561261e+05,4.389352e+06,0.003,0.0027,0.0019,10002
1,10002S002,Grasse (OCA),SLR,7845,4.581692e+06,5.561964e+05,4.389355e+06,0.0007,0.0007,0.0007,10002
...,...,...,...,...,...,...,...,...,...,...,...
1340,97401M003,La Reunion,GNSS,REUN,3.364099e+06,4.907945e+06,-2.293467e+06,0.0031,0.004,0.0025,97401
197,97401S003,La Reunion,DORIS,REUC,3.364080e+06,4.907924e+06,-2.293546e+06,0.0137,0.0097,0.0086,97401
1341,97413M001,Le Port,GNSS,LEPO,3.393377e+06,4.897825e+06,-2.267723e+06,0.0096,0.013,0.0067,97413
1342,97415M001,"Saint-Denis, Le",GNSS,STDE,3.376868e+06,4.912498e+06,-2.260738e+06,0.0111,0.0156,0.0081,97415


In [6]:
#Leave only station_ids, that occure more than 2 times
station_counts = concatenated_files['station_id'].value_counts()
valid_station_id = station_counts[station_counts > 2].index
filtered_stations = concatenated_files[concatenated_files['station_id'].isin(valid_station_id)]
filtered_stations = filtered_stations.sort_values('station_id')
filtered_stations

,id,name,type,code,x,y,z,dx,dy,dz,station_id
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,0.002,0.0025,0.002,10002
3,10002M010,Grasse (OCA),GNSS,GRAC,4.581708e+06,5.561328e+05,4.389341e+06,0.001,0.0007,0.001,10002
2,10002M006,Grasse (OCA),GNSS,GRAS,4.581691e+06,5.561150e+05,4.389361e+06,0.0017,0.0009,0.0015,10002
1,10002S002,Grasse (OCA),SLR,7845,4.581692e+06,5.561964e+05,4.389355e+06,0.0007,0.0007,0.0007,10002
0,10002M003,Grasse (OCA),VLBI,7605,4.581697e+06,5.561261e+05,4.389352e+06,0.003,0.0027,0.0019,10002
...,...,...,...,...,...,...,...,...,...,...,...
193,97301S004,KOUROU,DORIS,KRUB,3.855261e+06,-5.049735e+06,5.630580e+05,0.0158,0.0139,0.0129,97301
195,97401S001,La Reunion,DORIS,REUA,3.364094e+06,4.907946e+06,-2.293482e+06,0.0069,0.0064,0.0046,97401
196,97401S002,La Reunion,DORIS,REUB,3.364093e+06,4.907945e+06,-2.293482e+06,0.008,0.007,0.0046,97401
197,97401S003,La Reunion,DORIS,REUC,3.364080e+06,4.907924e+06,-2.293546e+06,0.0137,0.0097,0.0086,97401


In [7]:
#Get the number of techniques for each station
technic_counts = filtered_stations.groupby('station_id')['type'].nunique()

#Get id of the stations, that have more than one technic
more_one_tech = technic_counts[technic_counts > 1]
final_stations = filtered_stations[filtered_stations['station_id'].isin(more_one_tech.index)]
number_of_stations = final_stations['station_id'].value_counts()
number_of_stations

40451    20
14201    17
40405    11
10503    11
92201    11
         ..
21601     3
41508     3
40439     3
40496     3
40104     3
Name: station_id, Length: 105, dtype: int64

In [8]:
final_stations

,id,name,type,code,x,y,z,dx,dy,dz,station_id
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,0.002,0.0025,0.002,10002
3,10002M010,Grasse (OCA),GNSS,GRAC,4.581708e+06,5.561328e+05,4.389341e+06,0.001,0.0007,0.001,10002
2,10002M006,Grasse (OCA),GNSS,GRAS,4.581691e+06,5.561150e+05,4.389361e+06,0.0017,0.0009,0.0015,10002
1,10002S002,Grasse (OCA),SLR,7845,4.581692e+06,5.561964e+05,4.389355e+06,0.0007,0.0007,0.0007,10002
0,10002M003,Grasse (OCA),VLBI,7605,4.581697e+06,5.561261e+05,4.389352e+06,0.003,0.0027,0.0019,10002
...,...,...,...,...,...,...,...,...,...,...,...
193,97301S004,KOUROU,DORIS,KRUB,3.855261e+06,-5.049735e+06,5.630580e+05,0.0158,0.0139,0.0129,97301
195,97401S001,La Reunion,DORIS,REUA,3.364094e+06,4.907946e+06,-2.293482e+06,0.0069,0.0064,0.0046,97401
196,97401S002,La Reunion,DORIS,REUB,3.364093e+06,4.907945e+06,-2.293482e+06,0.008,0.007,0.0046,97401
197,97401S003,La Reunion,DORIS,REUC,3.364080e+06,4.907924e+06,-2.293546e+06,0.0137,0.0097,0.0086,97401


2. Calculate for each station the polygon of its extent (convex envelope) and export it as a SHP file (convert geometry from EPSG:4978 to EPSG:4326 at the begining).

In [9]:
#Make a geodata frame from the locations of the sites with the given coordinates and transform it to CRS 4326
gdf_stations = gpd.GeoDataFrame(final_stations, geometry=gpd.points_from_xy(final_stations.x, final_stations.y, final_stations.z), crs="EPSG:4978") 
gdf_stations = gdf_stations.to_crs(4326)
gdf_stations

,id,name,type,code,x,y,z,dx,dy,dz,station_id,geometry
0,10002S018,Grasse (OCA),DORIS,GR3B,4.581680e+06,5.561665e+05,4.389372e+06,0.002,0.0025,0.002,10002,POINT Z (6.92123 43.75483 1323.70087)
3,10002M010,Grasse (OCA),GNSS,GRAC,4.581708e+06,5.561328e+05,4.389341e+06,0.001,0.0007,0.001,10002,POINT Z (6.92077 43.75449 1319.85842)
2,10002M006,Grasse (OCA),GNSS,GRAS,4.581691e+06,5.561150e+05,4.389361e+06,0.0017,0.0009,0.0015,10002,POINT Z (6.92058 43.75474 1319.30335)
1,10002S002,Grasse (OCA),SLR,7845,4.581692e+06,5.561964e+05,4.389355e+06,0.0007,0.0007,0.0007,10002,POINT Z (6.92158 43.75463 1323.34323)
0,10002M003,Grasse (OCA),VLBI,7605,4.581697e+06,5.561261e+05,4.389352e+06,0.003,0.0027,0.0019,10002,POINT Z (6.92070 43.75463 1318.64665)
...,...,...,...,...,...,...,...,...,...,...,...,...
193,97301S004,KOUROU,DORIS,KRUB,3.855261e+06,-5.049735e+06,5.630580e+05,0.0158,0.0139,0.0129,97301,POINT Z (-52.63978 5.09863 109.69563)
195,97401S001,La Reunion,DORIS,REUA,3.364094e+06,4.907946e+06,-2.293482e+06,0.0069,0.0064,0.0046,97401,POINT Z (55.57177 -21.20836 1561.91250)
196,97401S002,La Reunion,DORIS,REUB,3.364093e+06,4.907945e+06,-2.293482e+06,0.008,0.007,0.0046,97401,POINT Z (55.57177 -21.20836 1561.08916)
197,97401S003,La Reunion,DORIS,REUC,3.364080e+06,4.907924e+06,-2.293546e+06,0.0137,0.0097,0.0086,97401,POINT Z (55.57176 -21.20899 1561.14383)


In [10]:
#Leave only columns with station id, geometry and copy them to a new data frame
station_points = gdf_stations.filter(['station_id', 'geometry']).dissolve(by='station_id')
station_polygons = station_points.copy()
print(station_polygons)

                                                     geometry
station_id                                                   
10002       MULTIPOINT Z (6.92058 43.75474 1319.30335, 6.9...
10003       MULTIPOINT Z (1.48076 43.56077 207.09635, 1.48...
10004       MULTIPOINT Z (-4.50383 48.40787 104.42082, -4....
10077       MULTIPOINT Z (8.76246 41.92747 98.24128, 8.762...
10202       MULTIPOINT Z (-21.99518 64.15098 95.75501, -21...
...                                                       ...
92301       MULTIPOINT Z (-134.96492 -23.13026 82.21310, -...
92701       MULTIPOINT Z (166.41006 -22.26983 83.06194, 16...
92902       MULTIPOINT Z (-178.12100 -14.30778 85.73272, -...
97301       MULTIPOINT Z (-52.80597 5.25219 -25.77014, -52...
97401       MULTIPOINT Z (55.57172 -21.20822 1558.37142, 5...

[105 rows x 1 columns]


In [11]:
#Convert multipoints to geometries with the Convex_hull function
station_polygons.geometry = station_polygons.geometry.convex_hull
station_polygons

#Make sure to leave only polygons for shape files
polygons = station_polygons[station_polygons.geometry.type == 'Polygon']
polygons

# Save the polygons into a shapefile
polygons.to_file('polygons_for_project.shp')  

,geometry
station_id,
10002,"POLYGON Z ((6.92077 43.75449 1319.85842, 6.920..."
10003,"POLYGON Z ((1.48489 43.54962 210.79597, 1.4812..."
10004,"POLYGON Z ((-4.49659 48.38050 65.82489, -4.503..."
10077,"POLYGON Z ((8.76270 41.92739 96.80211, 8.76246..."
10202,"POLYGON Z ((-21.95549 64.13879 93.04831, -21.9..."
...,...
92301,"POLYGON Z ((-134.96483 -23.13035 80.64485, -13..."
92701,"POLYGON Z ((166.41020 -22.26985 83.04824, 166...."
92902,"POLYGON Z ((-178.12095 -14.30780 84.87284, -17..."


List for each station the images (between 2022/01/01 and 2022/09/30) that are covering the extent and export it as a SHP file:

For each station, write a <station_id>.json temporary file when you do the API request;
Merge all information in one SHP file after.

In [78]:
from datetime import datetime
import requests
from requests.auth import HTTPBasicAuth

# Configuration
USERNAME = "username"
PASSWORD = "password"
NB_ITEMS = 20 # You can't change it in fact...

def request_images(wkt_geometry, date_1, date_2):
    items = [] # Empty list to store return elements
    start_position = 0 # You will need a while loop
    url = (
        f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?"
        f"cloudCover=[0,10]"
        f"&startDate={date_1}T00:00:00Z&completionDate={date_2}T23:59:59Z"
        f"&geometry={wkt_geometry}"
    )
    # Request
    r = requests.get(url)#, auth=HTTPBasicAuth(USERNAME, PASSWORD))
    # If status_code is not 200, we have an issue
    if r.status_code == 200:
        data = r.json()
        i = 0
        while i < NB_ITEMS:
            if 'features' in data:
                items += data['features']
                i+=1       
        return items


[{'type': 'Feature',
  'id': '72d990e4-28b1-474d-8673-578472025aef',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[2.98128485298927, 49.6518125814149],
     [1.6142723681857, 49.644430073461],
     [1.64154622508021, 48.6570207750582],
     [2.51112204193742, 48.6616482187594],
     [2.56288577299079, 48.7732422703949],
     [2.63058257989595, 48.917953790727],
     [2.69867440422509, 49.0626798171847],
     [2.76734777666624, 49.2073675306132],
     [2.8367530153596, 49.3519651381024],
     [2.90649613934253, 49.4965862081612],
     [2.97617664214705, 49.6411876669432],
     [2.98128485298927, 49.6518125814149]]]},
  'properties': {'collection': 'SENTINEL-2',
   'status': 'ONLINE',
   'license': {'licenseId': 'unlicensed',
    'hasToBeSigned': 'never',
    'grantedCountries': None,
    'grantedOrganizationCountries': None,
    'grantedFlags': None,
    'viewService': 'public',
    'signatureQuota': -1,
    'description': {'shortName': 'No license'}},
   'productIdentifier': '

In [14]:
#Make a list of temporal json files
temp_json_files = []

#Perform the request for all polygons in the df
for geom in polygons.geometry:
    temp_json_file = request_images(geom, datetime(2022, 1, 1).date(), datetime(2022, 9, 30).date())
    temp_json_files.append(temp_json_file)

In [ ]:
#Check the number of requested images. 
len(temp_json_files)

105

In [17]:
#Check how to get the final coordinates from the output dictionary
temp_json_files[0]
temp_json_files[0][0]['geometry']['coordinates'][0]

[[6.53537600651285, 43.2720289149594],
 [6.53677232313667, 43.2382625533271],
 [7.8886830141923, 43.2593919105371],
 [7.87023728676149, 44.2478306839688],
 [6.85573170854089, 44.2316889568923],
 [6.80770452180834, 44.0916012054146],
 [6.7580017505525, 43.9450313739555],
 [6.70988060072912, 43.7980728382952],
 [6.66052015967677, 43.6515224445036],
 [6.61099005955164, 43.5050672681946],
 [6.56336414120084, 43.3580741142308],
 [6.53537600651285, 43.2720289149594]]

In [20]:
#Make a geodata frame from the requested images
output_list = []
i = 0
while i < len(temp_json_files):
    coordinates = temp_json_files[i][0]['geometry']['coordinates'][0]
    polygon_geom = Polygon(coordinates)
    output_list.append({'geometry': polygon_geom})
    i += 1
api_images = gpd.GeoDataFrame(output_list)

#Save the data frame to a shapefile
api_images.to_file("api_images.shp", driver='ESRI Shapefile')

api_images

,geometry
0,"POLYGON ((6.53538 43.27203, 6.53677 43.23826, ..."
1,"POLYGON ((0.49593 44.22596, 0.53677 43.23826, ..."
2,"POLYGON ((-4.21863 48.39357, -4.22702 48.74648..."
3,"POLYGON ((7.78352 42.44624, 7.80218 41.45752, ..."
4,"POLYGON ((-23.11307 64.88918, -23.04007 63.924..."
...,...
100,"POLYGON ((-134.83974 -22.60596, -135.00020 -22..."
101,"POLYGON ((165.96675 -21.70060, 165.97356 -22.6..."
102,"POLYGON ((-178.60390 -14.55535, -177.83730 -14..."
103,"POLYGON ((-53.50034 4.43056, -52.71429 4.43291..."
